In [3]:
import os, sys, numpy as np
import argparse
from time import time
from tqdm import tqdm

import tensorflow # needs to call tensorflow before torch, otherwise crush
from utils.Logger import Logger

import torch
import torch.nn as nn
from torch.autograd import Variable

from models import model_factory
from torch.utils.data import ConcatDataset

import torchvision
import matplotlib.pyplot as plt

from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler

import copy
import numpy as np

batch_size = 128
num_workers = 4
n_classes = 31
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
import numpy as np
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image


class JigsawDataset(data.Dataset):
    def __init__(self, data_path, txt_list, classes=1000):
        self.data_path = data_path
        self.names, _ = self.__dataset_info(txt_list)
        self.N = len(self.names)
        self.permutations = self.__retrive_permutations(classes)

        self.__image_transformer = transforms.Compose([
            transforms.Resize(256, Image.BILINEAR),
            transforms.CenterCrop(255)])
        self.__augment_tile = transforms.Compose([
            transforms.RandomCrop(64),
            transforms.Resize((75, 75), Image.BILINEAR),
            transforms.ColorJitter(0.3, 0.3, 0.3, 0.3),
            transforms.RandomGrayscale(0.3),
            transforms.ToTensor()
        ])

    def __getitem__(self, index):
        framename = self.data_path + '/' + self.names[index]

        img = Image.open(framename).convert('RGB')
        img = self.__image_transformer(img)

        w = float(img.size[0]) / 3
        tiles = [None] * 9
        for n in range(9):
            y = int(n / 3)
            x = n % 3
            tile = img.crop([x * w, y * w, (x+1)*w, (y+1)*w])
            tile = self.__augment_tile(tile)
            # Normalize the patches indipendently to avoid low level features shortcut
            tiles[n] = tile 

        order = np.random.randint(len(self.permutations))
        data = [tiles[self.permutations[order][t]] for t in range(9)]
        data = torch.stack(data, 0)
        return torchvision.utils.make_grid(data, 3, padding=False), int(order)   

    def __len__(self):
        return len(self.names)

    def __dataset_info(self, txt_labels):
        with open(txt_labels, 'r') as f:
            images_list = f.readlines()

        file_names = []
        labels = []
        for row in images_list:
            row = row.split(' ')
            file_names.append(row[0])
            labels.append(int(row[1]))

        return file_names, labels

    def __retrive_permutations(self, classes):
        all_perm = np.load('permutations_%d.npy' % (classes))
        # from range [1,9] to [0,8]
        if all_perm.min() == 1:
            all_perm = all_perm - 1

        return all_perm

In [5]:
office_list = ["amazon","dslr","webcam"]
list_paths = {dataset: "/home/enoon/data/images/office/%s/train.txt" % dataset for dataset in office_list}
datasets = {dataset: JigsawDataset("", list_paths[dataset], n_classes) for dataset in office_list}
source = "amazon"
target = "dslr"
setting = [source,target]
# train_datasets = ConcatDataset([datasets[dataset] for dataset in setting])
# len(train_datasets)

In [6]:
dataloaders = {"train": torch.utils.data.DataLoader(datasets[source], batch_size=batch_size, shuffle=True, 
                                               num_workers=num_workers, pin_memory=True),
           "val": torch.utils.data.DataLoader(datasets[target], batch_size=batch_size, shuffle=False, 
                                               num_workers=num_workers, pin_memory=True)}
dataset_sizes = {"train": len(datasets[source]),
                "val": len(datasets[target])}

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    training_memory = {'train': np.zeros(num_epochs),
                      'val': np.zeros(num_epochs)}
    logger = Logger(num_epochs)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                print("Loss %.2f" % loss.item())

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            training_memory[phase][epoch] = epoch_acc
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, training_memory

In [12]:
model_ft = models.alexnet(pretrained=True)
classifier = nn.Sequential(model_ft.classifier[:6], nn.Linear(4096, n_classes))
model_ft.classifier = classifier
# print(model_ft)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, n_classes)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9, nesterov=True)
# optimizer_ft = optim.Adam(model_ft.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

In [13]:
model_ft, memory = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=4)

Epoch 0/3
----------
Loss 3.53
Loss 3.56
Loss 3.41
Loss 3.47
Loss 3.55
Loss 3.53
Loss 3.51
Loss 3.49
Loss 3.52
Loss 3.56
Loss 3.49
Loss 3.43
Loss 3.50
Loss 3.47
Loss 3.41
Loss 3.43
Loss 3.48
Loss 3.39
Loss 3.41
Loss 3.36
Loss 3.43
Loss 3.41
Loss 3.76
train Loss: 3.4705 Acc: 0.0373
Loss 3.49
Loss 3.53
Loss 3.50
Loss 3.50
val Loss: 3.5035 Acc: 0.0341

Epoch 1/3
----------
Loss 3.50
Loss 3.57
Loss 3.47
Loss 3.44
Loss 3.46
Loss 3.44
Loss 3.43
Loss 3.39
Loss 3.46
Loss 3.43
Loss 3.42
Loss 3.37
Loss 3.48
Loss 3.44
Loss 3.39
Loss 3.43
Loss 3.39
Loss 3.39
Loss 3.39
Loss 3.34
Loss 3.39
Loss 3.38
Loss 4.37
train Loss: 3.4276 Acc: 0.0511
Loss 3.36
Loss 3.39
Loss 3.39
Loss 3.37
val Loss: 3.3778 Acc: 0.0542

Epoch 2/3
----------
Loss 3.34
Loss 3.37
Loss 3.38
Loss 3.37
Loss 3.40
Loss 3.42
Loss 3.39
Loss 3.42
Loss 3.42
Loss 3.42
Loss 3.45
Loss 3.39
Loss 3.38
Loss 3.39
Loss 3.39
Loss 3.40
Loss 3.37
Loss 3.36
Loss 3.36
Loss 3.32
Loss 3.37
Loss 3.40
Loss 3.26
train Loss: 3.3868 Acc: 0.0472
Loss 3.34
Loss

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    return inp

conv1 = models.alexnet(pretrained=True).features[0] #model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
conv1 = model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
plt.plot(memory["train"], label="train")
plt.plot(memory["val"], label="val")
plt.show()

In [ ]:



# iter_c = iter(train_datasets)

# for x in range(5):
#     tmp = next(iter_c)
#     image = to_plt(tmp[0])
#     plt.imshow(image)
#     plt.show()

In [ ]:
image.shape

In [ ]:
resnet = models.resnet18(True)
tmp = resnet(torch.Tensor(image.reshape(1,3,225,225)))

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    return inp


iter_c = iter(c)

for x in range(5):
    tmp = next(iter_c)
    image = torchvision.utils.make_grid(tmp[2],3)
    plt.imshow(to_plt(image))
    plt.show()

In [ ]:
model_ft = models.alexnet(pretrained=True)
model_ft

In [ ]:
losses = {"ciao":3, "tu":1}
", ".join(["%s : %.2f" % (k, v) for k,v in losses.items()])